In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from decpomdp import DecPOMDP
import Classes
from constant import Constants
import time
import os



In [2]:
file_name = "dectiger"
game_type = "cooperative"
planning_horizon = 3
num_iterations = 2
sota_ = False


In [3]:
#import problem
problem = DecPOMDP(file_name, 1,horizon=planning_horizon)
# constant = Constants(problem)
# utilities = Utilities(constant)
Classes.set_problem(problem)

print(f"game of type {game_type} initiated with SOTA set to = {sota_} with horizon {planning_horizon}")
print(f"game size :\n\t|S| = {len(problem.states)}")
print(f"\t|Z| = {problem.num_joint_observations}\n\t|U| = {problem.num_joint_actions} with |U_i| = {problem.num_actions[0]}")
print(f"intiial_belief : {problem.b0}")


print(f"{problem.num_actions[0]},{problem.num_actions[0]}")

problem set to dectiger
game of type cooperative initiated with SOTA set to = False with horizon 3
game size :
	|S| = 2
	|Z| = 4
	|U| = 9 with |U_i| = 3
intiial_belief : [0.5 0.5]
3,3


In [4]:
# solve
def SOLVE(game):
    start_time = time.time()
    policy = game.solve(num_iterations,0.9)
    end_time = time.time()
    solve_time = end_time - start_time
    value_fn = game.value_function
    return policy,solve_time,value_fn

In [5]:
def initialize_database():
    database = {"gametype":[],
                "SOTA" : [],
                "horizon": [],
                    "num_iterations" : [],
                    "average_time" : [],
                    "number_of_beliefs" : [],
                    "ave_leader_value_b0":[],
                    "ave_follower_value_b0":[]
                    # "density" = []
                    # "gap":[]
                   
                    }
    return database

def add_to_database(database,horizon,game_type,num_iterations,average_time,num_beliefs,V0_B0,V1_B0,SOTA):
    sota = {True:"State of the Art" , False:"Stackelberg"}
    database["gametype"].append(game_type)
    database["horizon"].append(horizon)
    database["SOTA"].append(sota[SOTA])
    database["num_iterations"].append(num_iterations)
    database["average_time"].append(average_time)
    database["number_of_beliefs"].append(num_beliefs)
    database["ave_leader_value_b0"].append(V0_B0)
    database["ave_follower_value_b0"].append(V1_B0)
    # database["gap"].append(abs(V0_B0-V1_B0))
    # database["density"].append(density)
    return


In [6]:
database = initialize_database()
for gametype in ["stackelberg","cooperative","zerosum"]:
    for sota in [False,True]:
        for horizon in range(1,planning_horizon+1):
            print(f"\n===== GAME WITH HORIZON {horizon} , SOTA {sota} =====")
            problem = DecPOMDP(file_name,1,horizon=planning_horizon)
            Classes.set_problem(problem)
            game = Classes.PBVI(problem=problem,horizon=horizon,density=0.1,gametype=gametype,sota=sota)
            policy, time_ , value_fn = SOLVE(game)
            num_beliefs = game.belief_space.belief_size()
            value0,value1= value_fn.get_values_initial_belief()
            add_to_database(database,horizon,game_type,2,time_,num_beliefs,np.average(value0),np.average(value1),sota)
#POLICY PRINTING: 
# print("\nLEADER POLICY\n")
# policy[0].print_trees()
# print("\nFOLLOWER POLICY\n")
# policy[1].print_trees()        



database = pd.DataFrame(database)
path = "Results/"
file_name = f"{file_name}_{game_type}_{horizon}_experiment_results.csv"
database.to_csv(path+file_name, index=False)




===== GAME WITH HORIZON 1 , SOTA False =====
problem set to dectiger
	belief expansion done
iteration : 1


	backup at timestep 1 done
	backup at timestep 0 done
	backward induction done
iteration : 2
	backup at timestep 1 done
	backup at timestep 0 done
	backward induction done

===== GAME WITH HORIZON 2 , SOTA False =====
problem set to dectiger
	belief expansion done
iteration : 1
	backup at timestep 2 done
	backup at timestep 1 done
	backup at timestep 0 done
	backward induction done
iteration : 2
	backup at timestep 2 done
	backup at timestep 1 done
	backup at timestep 0 done
	backward induction done

===== GAME WITH HORIZON 3 , SOTA False =====
problem set to dectiger
	belief expansion done
iteration : 1
	backup at timestep 3 done
	backup at timestep 2 done
	backup at timestep 1 done
	backup at timestep 0 done
	backward induction done
iteration : 2
	backup at timestep 3 done
	backup at timestep 2 done
	backup at timestep 1 done
	backup at timestep 0 done
	backward induction done

===== GAME WITH HORIZON 1 , SOTA True =====
problem set to dectiger
	belief expansion done
iteration : 1
	b

In [7]:
database

,gametype,SOTA,horizon,num_iterations,average_time,number_of_beliefs,ave_leader_value_b0,ave_follower_value_b0
0,cooperative,Stackelberg,1,2,0.449024,4,-4.0,-19.0
1,cooperative,Stackelberg,2,2,0.884385,7,-6.0,-28.5
2,cooperative,Stackelberg,3,2,1.427134,10,-8.0,-38.0
3,cooperative,State of the Art,1,2,0.396457,4,-4.0,-9.5
4,cooperative,State of the Art,2,2,0.809873,7,-6.0,-9.5
5,cooperative,State of the Art,3,2,1.307436,10,-8.0,-9.5
6,cooperative,Stackelberg,1,2,0.418118,4,-4.0,-4.0
7,cooperative,Stackelberg,2,2,0.803711,7,-6.0,-6.0
8,cooperative,Stackelberg,3,2,1.410262,10,-8.0,-8.0
9,cooperative,State of the Art,1,2,0.007101,4,-4.0,-4.0


In [8]:
print(f"RESULTS FOR {file_name}:\n")

RESULTS FOR dectiger_cooperative_3_experiment_results.csv:

